# Test Data Prep
This page is meant to prepare test fixtures for use in hydrofunctions.

**Table of Contents:**
- Create sample NWIS responses to requests.
- Check values of test fixtures.
- Check individual values within the mult_flags fixture

In [1]:
import hydrofunctions as hf
print("Hydrofunctions version: ", hf.__version__)
import numpy as np
print("Numpy version: ", np.__version__)
import requests
print("Requests version: ", requests.__version__)

Hydrofunctions version:  0.1.7
Numpy version:  1.16.1
Requests version:  2.21.0


## Create sample NWIS responses to requests.

The cells below will generate the output a user would expect to get from successful requests by hf.NWIS.json().
In other words, these are not the raw JSON returned by the NWIS, but rather
the output from calling the .json() method on a Requests object. It actually
isn't json, but rather a Python dict. You can tell because the true and false 
in the original json have been coverted to True and False for Python.

Each of the fixtures will represent the response to a different type of request to the NWIS. These will be used
to test whether hydrofunctions is parsing the JSON correctly. Note that I need to call these directly from the NWIS
using Requests, not hydrofunctions!

**NOTE:** within the response from NWIS, there is a field called 'queryURL', which contains the query used to generate this response... except with one tiny modification. The 'queryURL' doesn't contain the '?' neccessary between the static part of the URL and the requested parameters. Is this a bug?

| Request Name     | NWIS request URL     |  Description                 |
|------------------|----------------------|------------------------------|
| JSON15min2month  | http://nwis.waterservices.usgs.gov/nwis/iv/?format=json%2C1.1&sites=03213700&parameterCd=00060&startDT=2016-09-01&endDT=2017-11-01             | A fairly long request for discharge from one site.  |
|two_sites_two_params_iv | http://waterservices.usgs.gov/nwis/iv/?format=json&sites=01541000,01541200&siteStatus=all  | Returns stage & discharge for two sites, along with qualifier flags. |
| nothing_avail  | http://waterservices.usgs.gov/nwis/iv/?format=json&indent=on&stateCd=al&parameterCd=00001&siteStatus=all   | This is basically an empty dataset. NWIS returns this in response to filters that filter too much stuff, requests for parameters that don't exist, or don't exist at this station... |
| mult_flags  | http://waterservices.usgs.gov/nwis/iv/?format=json&sites=01542500&startDT=2019-01-24&endDT=2019-01-28&parameterCd=00060   | NWIS returns data for discharge at one site, but with 3 _qualifier flags: 'e':estimated, 'P': provisional data, 'Ice'. |

In [2]:
header = {'Accept-encoding': 'gzip','max-age': '120'}
JSON15min2day_req =           requests.get("https://waterservices.usgs.gov/nwis/iv/?format=json%2C1.1&sites=03213700&parameterCd=00060&startDT=2016-09-01&endDT=2016-09-02", headers=header)
two_sites_two_params_iv_req = requests.get("https://waterservices.usgs.gov/nwis/iv/?format=json&sites=01541000,01541200&period=P1D", headers=header)
nothing_avail_req =           requests.get("https://waterservices.usgs.gov/nwis/iv/?format=json&indent=on&stateCd=al&parameterCd=00001", headers=header)
mult_flags_req =              requests.get("https://waterservices.usgs.gov/nwis/iv/?format=json&sites=01542500&startDT=2019-01-24&endDT=2019-01-28&parameterCd=00060", headers=header)

In [3]:
JSON15min2day_req.json()

{'name': 'ns1:timeSeriesResponseType',
 'declaredType': 'org.cuahsi.waterml.TimeSeriesResponseType',
 'scope': 'javax.xml.bind.JAXBElement$GlobalScope',
 'value': {'queryInfo': {'queryURL': 'http://nwis.waterservices.usgs.gov/nwis/iv/format=json%2C1.1&sites=03213700&parameterCd=00060&startDT=2016-09-01&endDT=2016-09-02',
   'criteria': {'locationParam': '[ALL:03213700]',
    'variableParam': '[00060]',
    'timeParam': {'beginDateTime': '2016-09-01T00:00:00.000',
     'endDateTime': '2016-09-02T23:59:59.000'},
    'parameter': []},
   'note': [{'value': '[ALL:03213700]', 'title': 'filter:sites'},
    {'value': '[mode=RANGE, modifiedSince=null] interval={INTERVAL[2016-09-01T00:00:00.000-04:00/2016-09-02T23:59:59.000Z]}',
     'title': 'filter:timeRange'},
    {'value': 'methodIds=[ALL]', 'title': 'filter:methodId'},
    {'value': '2019-02-18T20:26:09.666Z', 'title': 'requestDT'},
    {'value': '6d44ff10-33bb-11e9-a754-3440b59d3362', 'title': 'requestId'},
    {'value': 'Provisional data

In [4]:
two_sites_two_params_iv_req.json()

{'name': 'ns1:timeSeriesResponseType',
 'declaredType': 'org.cuahsi.waterml.TimeSeriesResponseType',
 'scope': 'javax.xml.bind.JAXBElement$GlobalScope',
 'value': {'queryInfo': {'queryURL': 'http://waterservices.usgs.gov/nwis/iv/format=json&sites=01541000,01541200&period=P1D',
   'criteria': {'locationParam': '[ALL:01541000, ALL:01541200]',
    'variableParam': 'ALL',
    'parameter': []},
   'note': [{'value': '[ALL:01541000, ALL:01541200]', 'title': 'filter:sites'},
    {'value': '[mode=PERIOD, period=P1D, modifiedSince=null]',
     'title': 'filter:timeRange'},
    {'value': 'methodIds=[ALL]', 'title': 'filter:methodId'},
    {'value': '2019-02-18T20:26:10.266Z', 'title': 'requestDT'},
    {'value': '6da08c90-33bb-11e9-9463-6cae8b6642f6', 'title': 'requestId'},
    {'value': 'Provisional data are subject to revision. Go to http://waterdata.usgs.gov/nwis/help/?provisional for more information.',
     'title': 'disclaimer'},
    {'value': 'caas01', 'title': 'server'}]},
  'timeSeries'

In [5]:
nothing_avail_req.json()

{'name': 'ns1:timeSeriesResponseType',
 'declaredType': 'org.cuahsi.waterml.TimeSeriesResponseType',
 'scope': 'javax.xml.bind.JAXBElement$GlobalScope',
 'value': {'queryInfo': {'queryURL': 'http://waterservices.usgs.gov/nwis/iv/format=json&indent=on&stateCd=al&parameterCd=00001',
   'criteria': {'locationParam': '[]',
    'variableParam': '[00001]',
    'parameter': []},
   'note': [{'value': '[al]', 'title': 'filter:stateCd'},
    {'value': '[mode=LATEST, modifiedSince=null]',
     'title': 'filter:timeRange'},
    {'value': 'methodIds=[ALL]', 'title': 'filter:methodId'},
    {'value': '2019-02-18T20:26:10.981Z', 'title': 'requestDT'},
    {'value': '6dfd79a0-33bb-11e9-9463-6cae8b6642f6', 'title': 'requestId'},
    {'value': 'Provisional data are subject to revision. Go to http://waterdata.usgs.gov/nwis/help/?provisional for more information.',
     'title': 'disclaimer'},
    {'value': 'caas01', 'title': 'server'}]},
  'timeSeries': []},
 'nil': False,
 'globalScope': True,
 'typeSu

In [6]:
mult_flags_req.json()

{'name': 'ns1:timeSeriesResponseType',
 'declaredType': 'org.cuahsi.waterml.TimeSeriesResponseType',
 'scope': 'javax.xml.bind.JAXBElement$GlobalScope',
 'value': {'queryInfo': {'queryURL': 'http://waterservices.usgs.gov/nwis/iv/format=json&sites=01542500&startDT=2019-01-24&endDT=2019-01-28&parameterCd=00060',
   'criteria': {'locationParam': '[ALL:01542500]',
    'variableParam': '[00060]',
    'timeParam': {'beginDateTime': '2019-01-24T00:00:00.000',
     'endDateTime': '2019-01-28T23:59:59.000'},
    'parameter': []},
   'note': [{'value': '[ALL:01542500]', 'title': 'filter:sites'},
    {'value': '[mode=RANGE, modifiedSince=null] interval={INTERVAL[2019-01-24T00:00:00.000-05:00/2019-01-28T23:59:59.000Z]}',
     'title': 'filter:timeRange'},
    {'value': 'methodIds=[ALL]', 'title': 'filter:methodId'},
    {'value': '2019-02-18T20:26:11.592Z', 'title': 'requestDT'},
    {'value': '6e6ae170-33bb-11e9-9463-6cae8b6642f6', 'title': 'requestId'},
    {'value': 'Provisional data are subjec

## Check values of test fixtures
**Several properties of the test dataframes can be checked, including: **
- .shape : the number of columns and rows. (returns a tuple)
- .columns.values : the names of the columns (returns a list)
- .index.is_unique : are there any repeated values in the time index values? (True|False)
- .index.is_monotonic : index values increase in order. (True|False)


In [7]:
# Import the 'JSON' dicts from our test_data module.
from tests.test_data import JSON15min2day, two_sites_two_params_iv, nothing_avail, mult_flags

In [8]:
two =  hf.extract_nwis_df(two_sites_two_params_iv)
json = hf.extract_nwis_df(JSON15min2day)
mult = hf.extract_nwis_df(mult_flags)

In [9]:
two.shape

(93, 8)

In [10]:
mult.columns.values

array(['USGS:01542500:00060:00000',
       'USGS:01542500:00060:00000_qualifiers'], dtype=object)

In [11]:
json.index.is_unique

True

In [12]:
two.index.is_monotonic

True

## Check individual values within the mult_flags fixture
The mult_flags dataframe should have all three flags, with multiple flags in some locations.
- 2019-01-24T10:30 has P & e
- 2019-01-28T16:00 has P & Ice; value is -999999; should be converted to np.nan


In [13]:
mult.loc['2019-01-24T10:30', 'USGS:01542500:00060:00000_qualifiers']

'P,e'

In [14]:
mult.loc['2019-01-28T16:00:00.000-05:00', 'USGS:01542500:00060:00000_qualifiers']

'P,Ice'

In [15]:
type(mult.loc['2019-01-28T16:00:00.000-05:00', 'USGS:01542500:00060:00000'])

numpy.float64

In [16]:
mult.loc['2019-01-28T16:00:00.000-05:00', 'USGS:01542500:00060:00000']

nan